In [ ]:
#imports
import cv2
from imutils import paths
import numpy as np
import math
import pandas as pd
import re

In [ ]:
def vid2frame(input_path,output_path):
    """Takes in a video and splits into frames; Returns number of frames created"""
    
    capture = cv2.VideoCapture(input_path)
    frameNr = 0
    
    totalFrames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
    width  = capture.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
    
    
    
    while capture.isOpened():
        ret, frame = capture.read()
        centreW = round(width/2)
        centreH = round(height/2)
        
        #if frame is read correctly ret is True
        if not ret:
            #print("Can't receive frame (stream end?). Exiting ...")
            break
        else:
            rotate_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)          # Rotate frame
            crop_frame = rotate_frame[centreW-100:centreW+100,centreH-100:centreH+100]
        
            frame_name = 'frame_'+str(frameNr)+'.jpg'
            save_path = output_path +'/'+frame_name
            cv2.imwrite(save_path,crop_frame)
            frameNr = frameNr + 1

    return totalFrames
        

In [ ]:
def getFileLabels(path,dataNames = ['frame1.jpeg','frame2.jpeg']):
    
    video_name = path.split('/')[-1]
    label = video_name.split('_')
    
    t = int(label[2].split('=')[-1])
    m = int(label[3].split('=')[-1])
    x = float(label[4].split('=')[-1])
    q = label[5].split('=')[-1]
    flow_type =  label[7].split('.')[0]

    data = {'filename': dataNames,
        't': t,
        'm': m,
        'x': x,
        'q': q,
        'flow':flow_type}

    return pd.DataFrame(data)

In [ ]:
# def frame2strip(input_path,strip_size,strip_x_location,output_frame_length,frame_name_start = 0):
#     """Crops a full image into a strip"""
#     frameNr =0
#     out_frameNr = frame_name_start

#     frame_names = sorted_alphanumeric(os.listdir(input_path))
#     frames_num = len(frame_names)  #number of frames in the video/directory
    

#     possible_full_frames = math.floor(frames_num/(output_frame_length/strip_size))
#     print(f'Can create {possible_full_frames} frames from video {frames_num}')

#     #List to store generated file names
#     dataNames = []


#     for name in frame_names:
#         path = input_path +'/'+ name
#         frame = cv2.imread(path)
#         strip = frame[:, strip_x_location:strip_x_location +strip_size]

#         if frameNr == 0 :
#             output_frame = strip
#             frameNr = frameNr +1
#         else :
#             output_frame = np.concatenate((output_frame,strip),axis=1)
#             frameNr = frameNr +1

#         if(frameNr == output_frame_length/strip_size):
#             name = 'frame_'+str(out_frameNr)+'.jpg'
#             cv2.imwrite(f'D:\Flow Videos\dataset5\{name}', output_frame)
#             dataNames.append(name) 
#             out_frameNr = out_frameNr + 1 # change back to 1
#             frameNr = 0

#             if(out_frameNr == possible_full_frames + frame_name_start ):
#                 break

    
#     return out_frameNr,dataNames

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
#Location of raw data
input_path = r'D:\Flow Videos\raw_data\regime_videos\\'
frame_output_path = r'D:\Flow Videos\full_frames\regime_videos\\'

regime_type_paths = os.listdir(input_path)
regime_types = [ i.split('_')[-1] for i in regime_type_paths ] #regime types
print(regime_types)

video_paths = []
frame_output_paths = []

for i in regime_type_paths:
    in_path = input_path + i
    out_path = frame_output_path + i

    video_names = os.listdir(in_path)
    
    video_paths.extend((in_path +'/'+ i) for i in video_names )
    frame_output_paths.extend((out_path +'/'+ i) for i in video_names )

#video_paths

In [ ]:
# #Split all videos into frames

# for i in range(len(video_paths)):
#     vid_path = video_paths[i]
#     out_path = frame_output_paths[i]

#     os.makedirs(out_path, exist_ok=True)
#     vid2frame(vid_path,out_path)
#     print(f"Video {i} done")

In [ ]:
#Parameters
strip_size = 10    # Width of strip in pixels
frames_req =  2500 - 1   # number of frames used (generates 1 extra hence -1)
strip_x_location = 1     # Horizontal loaction of strip start in pixels
output_frame_length = 1000  # Size of Output frame in pixels

In [ ]:
input_path = frame_output_paths[1]
print(input_path)

In [ ]:

frameNr =0
out_frameNr = 0

frame_names = sorted_alphanumeric(os.listdir(input_path))
frames_num = len(frame_names)  #number of frames in the video/directory
frame_names

In [ ]:


possible_full_frames = math.floor(frames_num/(output_frame_length/strip_size))
print(f'Can create {possible_full_frames} frames from video {frames_num}')

#List to store generated file names
dataNames = []


for name in frame_names:
    path = input_path +'/'+ name
    frame = cv2.imread(path)
    strip = frame[:, strip_x_location:strip_x_location +strip_size]

    if frameNr == 0 :
        output_frame = strip
        frameNr = frameNr +1
    else :
        output_frame = np.concatenate((strip,output_frame),axis=1)
        frameNr = frameNr +1

    if(frameNr == output_frame_length/strip_size):
        name = 'frame_'+str(out_frameNr)+'.jpg'
        cv2.imwrite(f'D:\Flow Videos\dataset5\{name}', output_frame)
        dataNames.append(name) 
        out_frameNr = out_frameNr + 1 # change back to 1
        frameNr = 0



    


In [ ]:
frame_name_start = 0
labels = []
count =0
for i in range(len(frame_output_paths)):
    input_path = frame_output_paths[i]
    print(count)
    count =count +1
    
    frame_name_start,dataNames = frame2strip(input_path,strip_size,strip_x_location,output_frame_length,frame_name_start)

    labels.append(getFileLabels(input_path,dataNames))

labels =pd.concat(labels)
#print(labels)    

In [ ]:
labels.to_csv('D:\Flow Videos/data_labels_4.csv',index=False)